In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats

In [2]:
stocks = pd.read_csv('../sp_500_stocks.csv')

In [4]:
from secrets import IEX_CLOUD_API_TOKEN

In [16]:
def get_stock_stats(symbol):
    api = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'

In [8]:
batches = [stocks[:86]['Ticker'], stocks[86: 192]['Ticker'], stocks[192:277]['Ticker'], stocks[277: 359]['Ticker'], stocks[359: 444]['Ticker'], stocks[444:]['Ticker']]

In [44]:
cols = ['Ticker', 'Price', 'One-Year Price Return', '#Shares to buy']
dictionary = {col: list() for col in cols}

In [45]:
def batch_calls(tickers):
    tickers = ','.join(list(tickers))
    batch_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={tickers}&types=stats,price&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_url)
    if data.status_code != 200:
        return None
    return data.json()

In [46]:
for batch in batches:
    data = batch_calls(batch)
    if data:
        
        for ticker in batch:
            if ticker in data:
                dictionary['Ticker'].append(ticker)
                dictionary['Price'].append(data[ticker]['price'])
                dictionary['One-Year Price Return'].append(data[ticker]['stats']['year1ChangePercent'])
                dictionary['#Shares to buy'].append('N/A')

In [77]:
df = pd.DataFrame(dictionary)

In [78]:
# goal is to arrange top 50 highest year1ChangePercent --> high momentum
df.sort_values('One-Year Price Return', ascending= False, ignore_index=True, inplace = True)

In [85]:
portfolio_size = 10e7
cap_alloc = 10e7/50
df = df.iloc[:50]
df['#Shares to buy'] = cap_alloc//df['Price']

In [86]:
df

,Ticker,Price,One-Year Price Return,#Shares to buy
0,LB,83.850,2.316093,23852.0
1,DVN,59.810,1.286576,33439.0
2,OXY,57.700,1.003496,34662.0
3,MRO,23.200,0.897219,86206.0
4,APA,38.850,0.881627,51480.0
5,CF,92.990,0.830104,21507.0
6,MOS,52.160,0.717578,38343.0
7,COP,93.200,0.706484,21459.0
8,MCK,318.500,0.653562,6279.0
9,PXD,236.400,0.598980,8460.0


# Sustainable Momentum Strategy

We'll analyze the steady return rather than just a one year return. The goal is to find companies that have steady growth rather than "explosive growth"

Going to use One-Month, Three-Month, Six-Month, One-Year

In [94]:
cols = [
    'Ticker', 
    'Price', 
    'One-Month PR', 
    'One-Month PR Percentile',
    'Three-Month PR', 
    'Three-Month PR Percentile',
    'Six-Month PR', 
    'Six-Month PR Percentile',
    'One-Year PR', 
    'One-Year PR Percentile',
    '#Shares to buy'
]
dictionary = {col: list() for col in cols}

In [95]:
for batch in batches:
    data = batch_calls(batch)
    if data:
        
        for ticker in batch:
            if ticker in data:
                dictionary['Ticker'].append(ticker)
                dictionary['Price'].append(data[ticker]['price'])
                dictionary['One-Month PR'].append(data[ticker]['stats']['month1ChangePercent'])
                dictionary['Three-Month PR'].append(data[ticker]['stats']['month3ChangePercent'])
                dictionary['Six-Month PR'].append(data[ticker]['stats']['month6ChangePercent'])
                dictionary['One-Year PR'].append(data[ticker]['stats']['year1ChangePercent'])
                dictionary['#Shares to buy'].append('N/A')
                dictionary['One-Month PR Percentile'].append('N/A')
                dictionary['Three-Month PR Percentile'].append('N/A')
                dictionary['Six-Month PR Percentile'].append('N/A')
                dictionary['One-Year PR Percentile'].append('N/A')

In [96]:
df = pd.DataFrame(dictionary)

In [98]:
def get_percentile(period):
    sz = df[period].size-1
    col = period + " Percentile"
    df[col] = df[period].rank(method='max').apply(lambda x: 100.0 * (x-1)/sz)

In [102]:
periods = ["One-Month PR", 'Three-Month PR', 'Six-Month PR', 'One-Year PR']

In [103]:
for period in periods:
    get_percentile(period)

In [104]:
df

,Ticker,Price,One-Month PR,One-Month PR Percentile,Three-Month PR,Three-Month PR Percentile,Six-Month PR,Six-Month PR Percentile,One-Year PR,One-Year PR Percentile,#Shares to buy
0,A,115.80,-0.092009,24.898785,-0.176942,38.461538,-0.269624,26.113360,-0.225015,26.518219,N/A
1,AAL,13.70,-0.200052,2.226721,-0.192682,31.781377,-0.283916,23.886640,-0.429805,5.465587,N/A
2,AAP,179.60,-0.049909,51.821862,-0.162080,43.927126,-0.245178,32.186235,-0.113527,46.761134,N/A
3,AAPL,140.40,-0.012794,73.481781,-0.180672,37.044534,-0.221044,38.663968,0.032937,72.469636,N/A
4,ABBV,153.52,-0.051358,50.404858,-0.097789,62.955466,0.130510,89.271255,0.308156,92.914980,N/A
...,...,...,...,...,...,...,...,...,...,...,...
490,YUM,116.00,0.001891,81.376518,-0.056514,75.101215,-0.157976,51.012146,-0.028172,60.121457,N/A
491,ZBH,109.13,-0.135389,7.894737,-0.155877,46.153846,-0.165981,49.797571,-0.388375,8.299595,N/A
492,ZBRA,307.65,-0.091438,25.506073,-0.319625,5.668016,-0.499906,1.821862,-0.417353,6.477733,N/A
493,ZION,55.02,0.020720,87.246964,-0.222008,22.267206,-0.142245,53.643725,0.006759,69.230769,N/A


In [108]:
# calculate mean percentile --> average of how high they ranked in comparison
df['Avg Percentile'] = (df['One-Month PR Percentile'] + df['Three-Month PR Percentile'] + df['Six-Month PR Percentile'] + df['One-Year PR Percentile'])/4

In [112]:
df.sort_values('Avg Percentile', ascending= False, inplace=True, ignore_index=True)

In [113]:
df

,Ticker,Price,One-Month PR,One-Month PR Percentile,Three-Month PR,Three-Month PR Percentile,Six-Month PR,Six-Month PR Percentile,One-Year PR,One-Year PR Percentile,#Shares to buy,Avg Percentile
0,LB,83.25,0.082982,96.153846,0.215656,98.178138,0.823222,98.987854,2.354187,99.190283,N/A,98.127530
1,COG,22.66,0.261741,98.987854,0.263900,98.785425,0.181684,91.902834,0.342730,93.522267,N/A,95.799595
2,HRB,35.51,0.029601,90.080972,0.318197,98.987854,0.472122,97.570850,0.493032,95.951417,N/A,95.647773
3,VRTX,281.51,0.046186,93.724696,0.082814,95.344130,0.244703,93.522267,0.449737,94.939271,N/A,94.382591
4,DLTR,158.98,0.225544,98.785425,0.009602,88.259109,0.150981,91.093117,0.552811,96.558704,N/A,93.674089
...,...,...,...,...,...,...,...,...,...,...,...,...
490,CCL,9.82,-0.274934,0.202429,-0.502503,0.607287,-0.550862,1.012146,-0.672351,0.404858,N/A,0.556680
491,CTL,11.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A,NaN
492,ETFC,51.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A,NaN
493,MYL,15.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A,NaN
